# Code, Discover, Improve

### Note 1) Using ChatGPT4. Tried it with 3.5, and the code generation from the first ask was not usable.

### Note 2) This is not an executable Jupyter Notebook. Just follow along.

### Goal: We'll demonstrate how to get started with code, do needed exploration, then patch the original code with what we discover. The imortant part: This all happens in ChatGPT. We won't be manually updating the code we start with. ChatGPT does the patching for us.

Summary Steps:
1) We'll put together a set of instructions and ask chatGPT for code.
2) Find out the code is not good enough.
3) Do some more asking.
4) Finish with a telling for a code update.

 ### ASK THIS of CHATGPT. Keep the hashes in the request. Don't be at all concerned with the purpose of the code or whether it works.

In [ ]:
# create a python program with a main function
# the main function will be called by the cloud function
# the main function will take a pubsub message as an argument
# the main program will extract the project, data-set, and table from landed table from the pubsub message
# the main function will print the pubsub message
# the main function will return a string
# when successful, the cloud function will return the string
# when unsuccessful, the cloud function will return an error message

### You'll get something like this

In [ ]:
import base64
import json

def main(event, context):
    try:
        # Get the Pub/Sub message from the event
        pubsub_message = event['data']
        
        # Decode the message from base64
        decoded_message = base64.b64decode(pubsub_message).decode('utf-8')
        
        # Load the decoded message as a JSON object
        message_json = json.loads(decoded_message)
        
        # Extract the project, data-set, and table from the JSON object
        project = message_json.get('project', '')
        data_set = message_json.get('data_set', '')
        table = message_json.get('table', '')
        
        # Print the extracted information
        print(f"Pub/Sub message: {decoded_message}")
        print(f"Project: {project}")
        print(f"Data Set: {data_set}")
        print(f"Table: {table}")
        
        # Return a success string
        return f"Successfully processed message: {decoded_message}"
        
    except Exception as e:
        # Return an error message if there's an issue
        return f"Error processing message: {str(e)}"

# If this file is being run as a script, call the main function with sample data
if __name__ == "__main__":
    sample_event = {
        'data': base64.b64encode(json.dumps({
            'project': 'sample_project',
            'data_set': 'sample_data_set',
            'table': 'sample_table'
        }).encode('utf-8')).decode('utf-8')
    }
    sample_context = None
    result = main(sample_event, sample_context)
    print(result)

Analysis: This code snippet is not good enough because the "Extract the project, data-set, and table from the JSON object" won't actually get me the table related info from the event data. It's my fault. Let's pretend that I didn't know to tell ChatGPT that the event was for the creation of a table in BigQuery. in GCP, when you create a table and listen out for it's creation (details aren't important), you can work with the JSON telling you all about that table creation. Let's do some more asking.

## Ask ChatGP:

What does a bigquery json field look like when a table is created in a particular data-set?

### You'll get something like this

{
  "protoPayload": {
    "@type": "type.googleapis.com/google.cloud.audit.AuditLog",
    "status": {},
    "authenticationInfo": {
      "principalEmail": "user@example.com"
    },
    "requestMetadata": {
      "callerIp": "1.2.3.4",
      "callerSuppliedUserAgent": "google-cloud-sdk",
      "requestAttributes": {},
      "destinationAttributes": {}
    },
    "serviceName": "bigquery.googleapis.com",
    "methodName": "google.cloud.bigquery.v2.TableService.InsertTable",
    "authorizationInfo": [
      {
        "permission": "bigquery.tables.create",
        "granted": true,
        "resource": "projects/sample-project/datasets/sample_dataset"
      }
    ],
    "resourceName": "projects/sample-project/datasets/sample_dataset/tables/sample_table",
    "request": {
      "tableReference": {
        "projectId": "sample-project",
        "datasetId": "sample_dataset",
        "tableId": "sample_table"
      },
      "schema": {
        "fields": [
          {
            "name": "id",
            "type": "INTEGER",
            "mode": "NULLABLE"
          },
          {
            "name": "name",
            "type": "STRING",
            "mode": "NULLABLE"
          }
        ]
      },
      "timePartitioning": {
        "type": "DAY"
      },
      "description": "Sample table description",
      "friendlyName": "Sample Table"
    },
    "response": {
      "@type": "type.googleapis.com/google.cloud.bigquery.v2.Table",
      "tableReference": {
        "projectId": "sample-project",
        "datasetId": "sample_dataset",
        "tableId": "sample_table"
      }
    }
  },
  "insertId": "abc123",
  "resource": {
    "type": "bigquery_resource",
    "labels": {
      "project_id": "sample-project",
      "dataset_id": "sample_dataset",
      "table_id": "sample_table"
    }
  },
  "timestamp": "2021-10-01T00:00:00.000Z",
  "severity": "NOTICE",
  "logName": "projects/sample-project/logs/cloudaudit.googleapis.com%2Factivity",
  "receiveTimestamp": "2021-10-01T00:00:00.000Z"
}


## Ask ChatGPT:

With python, write code to extract the project, dataset, and table name from the above json

### You'll get something like this

In [ ]:
# Writer's note: I'll skip the assignment of some long JSON string to a json_string variable and continue with the important code...

# Load the JSON string as a Python dictionary
json_data = json.loads(json_string)

# Extract the project, dataset, and table name from the JSON data
project = json_data['protoPayload']['request']['tableReference']['projectId']
dataset = json_data['protoPayload']['request']['tableReference']['datasetId']
table = json_data['protoPayload']['request']['tableReference']['tableId']

# The Cool Part

## Ask CHATGPT:

Use the code variable assignment code for project, dataset, and table you just generated to update the  program generated from the prompt with the #s.

### You'll get this, with the patched code for extracting the data we need from the JSON!

In [ ]:
import base64
import json

def main(event, context):
    try:
        # Get the Pub/Sub message from the event
        pubsub_message = event['data']
        
        # Decode the message from base64
        decoded_message = base64.b64decode(pubsub_message).decode('utf-8')
        
        # Load the decoded message as a JSON object
        message_json = json.loads(decoded_message)

        # Extract the project, dataset, and table name from the JSON data
        project = message_json['protoPayload']['request']['tableReference']['projectId']
        dataset = message_json['protoPayload']['request']['tableReference']['datasetId']
        table = message_json['protoPayload']['request']['tableReference']['tableId']
        
        # Print the extracted information
        print(f"Pub/Sub message: {decoded_message}")
        print(f"Project: {project}")
        print(f"Dataset: {dataset}")
        print(f"Table: {table}")
        
        # Return a success string
        return f"Successfully processed message: {decoded_message}"
        
    except Exception as e:
        # Return an error message if there's an issue
        return f"Error processing message: {str(e)}"

# If this file is being run as a script, call the main function with sample data
if __name__ == "__main__":
    sample_event = {
        'data': base64.b64encode(json.dumps({
            'protoPayload': {
                'request': {
                    'tableReference': {
                        'projectId': 'sample-project',
                        'datasetId': 'sample_dataset',
                        'tableId': 'sample_table'
                    }
                }
            }
        }).encode('utf-8')).decode('utf-8')
    }
    sample_context = None
    result = main(sample_event, sample_context)
    print(result)


### Conclusion

Here, I demonstrated a workflow that started with a not-quite complete description of what we needed and defintely did not setup ChatGPT for success. Incrementally, I worked myself towards a solution and then looped back to the orginal code request, referring to that code by the incidental usage of #s (which were from a Python Script where I was starting out with a comment block), and asked to patch the code with the new understanding of how to work with JSON that describes a table creation.

Observations:
1) The sample event in the '_main_' part was updated to reflect the relevant BigQuery JSON structure. I DID NOT DO THAT. ChatGPT did that.
2) When I tried these instructions in a separate ChatGPT session, I noticed that the variable names for project, dataset, and table name were different in the original request, compared to the request for working with a sample table creation JSON. Yet, when I asked for the update of the original code with the updated fragment, it had no problem updating the appropriate variable assignments AND still keep the original variable names! 